# Welcome

We are going to be gathering weather data from the past year and throwing it into an SQLite database.
I realized on v1 that I was just throwing away data. Not good.

In [88]:
import requests
import json
from datetime import datetime
import pandas as pd

In [2]:
key_path = 'C:/Users/asalm/My Drive (asalmon2001@gmail.com)/Github Repos/Secrets'
key_file = open(key_path + '/weatherapi.txt', 'r')
api_key = key_file.read()

Lets check what historical data we can get

In [3]:
params = { 'key':api_key , 'q':'Calgary', 'dt':'2023-09-14' }
sample_response = requests.get('http://api.weatherapi.com/v1/history.json', params=params)
print(sample_response.status_code)

200


In [4]:
print(json.dumps(obj=sample_response.json(), indent=2))

{
  "location": {
    "name": "Calgary",
    "region": "Alberta",
    "country": "Canada",
    "lat": 51.08,
    "lon": -114.08,
    "tz_id": "America/Edmonton",
    "localtime_epoch": 1695650508,
    "localtime": "2023-09-25 8:01"
  },
  "forecast": {
    "forecastday": [
      {
        "date": "2023-09-14",
        "date_epoch": 1694649600,
        "day": {
          "maxtemp_c": 22.7,
          "maxtemp_f": 72.9,
          "mintemp_c": 9.0,
          "mintemp_f": 48.2,
          "avgtemp_c": 14.6,
          "avgtemp_f": 58.3,
          "maxwind_mph": 6.7,
          "maxwind_kph": 10.8,
          "totalprecip_mm": 0.0,
          "totalprecip_in": 0.0,
          "avgvis_km": 9.7,
          "avgvis_miles": 6.0,
          "avghumidity": 56.0,
          "condition": {
            "text": "Sunny",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/113.png",
            "code": 1000
          },
          "uv": 6.0
        },
        "astro": {
          "sunrise": "07:10 AM",
  

Lets think about what tables we want.
1. Forecast_hourly_data
   Columns: hour_id (ie. 23091403), *all the columns in each hour (minus condition: that we will just keep the code)
3. Forecast_day_data
4. Astro_data
5. 

In [82]:
import sqlite3 as sl
connection = sl.connect('weather_v2_database.db')
print(connection.total_changes)

cursor = connection.cursor()

0


In [6]:
def conversion(element) :
    return '\'' + str(element) + '\''

# Table Creation

## Hourly Weather Table

In [93]:
# Produce all the column names from the json

# sample_hourly = sample_response.json()['forecast']['forecastday'][0]['hour']
# column_names = ['hour_id', 'day_id']
# key_value_list = list(sample_hourly[0].items())
# for key, value in key_value_list[1:5] :
#     column_names.append(key)
# for key, value in key_value_list[6:] :
#     column_names.append(key)
# column_names.append('condition_text')
# column_names.append('condition_code')

# df = pd.DataFrame(column_names)
# df.to_csv('hourly_weather_columns.csv')

In [101]:
def getHourlyWeatherColumnNames() :
    dataframe = pd.read_csv('hourly_weather_columns.csv')
    my_list = dataframe['0'].values.tolist()[:]
    return my_list

In [102]:
# print(getHourlyWeatherColumnNames())

['hour_id', 'day_id', 'time', 'temp_c', 'temp_f', 'is_day', 'wind_mph', 'wind_kph', 'wind_degree', 'wind_dir', 'pressure_mb', 'pressure_in', 'precip_mm', 'precip_in', 'humidity', 'cloud', 'feelslike_c', 'feelslike_f', 'windchill_c', 'windchill_f', 'heatindex_c', 'heatindex_f', 'dewpoint_c', 'dewpoint_f', 'will_it_rain', 'chance_of_rain', 'will_it_snow', 'chance_of_snow', 'vis_km', 'vis_miles', 'gust_mph', 'gust_kph', 'uv', 'condition_text', 'condition_code']


In [59]:
def deleteHourlyWeatherTable() :
    try:
        cursor.execute('DROP TABLE hourly_weather')
    except:
        print('TABLE hourly_weather DOES NOT EXIST')

In [60]:
# Printing the database
def printHourlyWeather() :
    rows = cursor.execute('SELECT * FROM hourly_weather').fetchall()
    print(rows)

In [103]:
# Create hourly_weather table
def startHourlyWeatherTable() :
    column_names = getHourlyWeatherColumnNames()
    command = 'CREATE TABLE hourly_weather ( ' + ', '.join(list(map(conversion, column_names))) + ')'
    
    deleteHourlyWeatherTable()
    cursor.execute(command)
    printHourlyWeather()

In [74]:
# This is how I will format the hour_id
def getIdsAndTime(hour) :
    my_datetime = datetime.fromisoformat(hour['time'])
    [day_id, hour_id, time] = my_datetime.strftime('%y%m%d-%y%m%d%H-%H:%M').split('-')
    return [hour_id, day_id, time]

In [75]:
def insertHourlyData(hour) :
    insertion_values = getIdsAndTime(hour)
    keys_list = list(hour.keys())
    values_list = list(hour.values())
    insertion_values = insertion_values + values_list[2:5]
    insertion_values = insertion_values + values_list[6:]
    insertion_values.append(values_list[5]['text'])
    insertion_values.append(values_list[5]['code'])
    command = 'INSERT INTO hourly_weather VALUES (' + ', '.join(list(map(conversion, insertion_values))) + ')'
    cursor.execute(command)

In [79]:
# sample_hourly = sample_response.json()['forecast']['forecastday'][0]['hour']
# #for hour in sample_hourly :
# #    insertHourlyData(hour)
# startHourlyWeatherTable()
# for hour in sample_hourly :
#     insertHourlyData(hour)

[]


We now have a way to insert an entire day of hourly data into the hourly_weather table

In [80]:
df = pd.read_sql_query('SELECT * FROM hourly_weather', connection)
df.tail()

,hour_id,day_id,time,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,wind_dir,...,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition_text,condition_code
19,23091419,230914,19:00,18.7,65.7,1,6.7,10.8,101,ESE,...,0,0,0,10.0,6.0,10.5,16.9,5.0,Sunny,1000
20,23091420,230914,20:00,15.3,59.5,0,6.3,10.1,97,E,...,0,0,0,10.0,6.0,13.2,21.2,1.0,Clear,1000
21,23091421,230914,21:00,13.9,57.0,0,4.5,7.2,91,E,...,0,0,0,10.0,6.0,9.4,15.1,1.0,Clear,1000
22,23091422,230914,22:00,12.5,54.5,0,3.4,5.4,79,E,...,0,0,0,10.0,6.0,6.9,11.2,1.0,Clear,1000
23,23091423,230914,23:00,11.6,52.9,0,2.2,3.6,77,ENE,...,0,0,0,10.0,6.0,4.7,7.6,1.0,Clear,1000


## Day Forecast

In [104]:
sample_day = sample_response.json()['forecast']['forecastday'][0]
print(sample_day)

{'date': '2023-09-14', 'date_epoch': 1694649600, 'day': {'maxtemp_c': 22.7, 'maxtemp_f': 72.9, 'mintemp_c': 9.0, 'mintemp_f': 48.2, 'avgtemp_c': 14.6, 'avgtemp_f': 58.3, 'maxwind_mph': 6.7, 'maxwind_kph': 10.8, 'totalprecip_mm': 0.0, 'totalprecip_in': 0.0, 'avgvis_km': 9.7, 'avgvis_miles': 6.0, 'avghumidity': 56.0, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'uv': 6.0}, 'astro': {'sunrise': '07:10 AM', 'sunset': '07:53 PM', 'moonrise': '06:32 AM', 'moonset': '08:04 PM', 'moon_phase': 'New Moon', 'moon_illumination': '1'}, 'hour': [{'time_epoch': 1694671200, 'time': '2023-09-14 00:00', 'temp_c': 13.4, 'temp_f': 56.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 351, 'wind_dir': 'N', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 54, 'cloud': 56, 'feelsli